# Liberar DNI - Notebook de pruebas

Este notebook requiere las credenciales de tu DNI para iniciar sesión en el portal de cita previa en un archivo `.env`.
Simplemente crea una copia de `.env.sample` llamada `.env` y rellena los datos con los que figuran en tu DNI.

In [1]:
import pandas as pd

import geopandas as gpd
import geopy

from geopy.geocoders import Nominatim


from selenium.webdriver import Chrome
from tqdm.auto import tqdm

from utils import (extraer_datos_unidad, extract_hours,
                   get_remaining_days, get_remaining_months, login, URL_LOGIN, URL_DNI, URL_PASAPORTE)

In [3]:
tqdm.pandas()

In [4]:
driver = Chrome()

driver.get(URL_LOGIN)

## Login

In [5]:
login(driver, tesseract_check=True)

# DNI Extract

## Provinces

In [6]:
driver.get(URL_DNI)

In [22]:
provinces = {}

map_elem = driver.find_element_by_id('Map')
for map_child in map_elem.find_elements_by_css_selector('*'):
    name = map_child.get_attribute('alt')
    url = map_child.get_attribute('href')
    
    provinces[name] = url

In [23]:
provinces

{'A Coruña': 'https://www.citapreviadnie.es/citaPreviaDniExp/ObtenerOficinas.action?provincia=15&nombre_provincia=A%20Coru%C3%B1a',
 'Lugo': 'https://www.citapreviadnie.es/citaPreviaDniExp/ObtenerOficinas.action?provincia=27&nombre_provincia=Lugo',
 'Ourense': 'https://www.citapreviadnie.es/citaPreviaDniExp/ObtenerOficinas.action?provincia=32&nombre_provincia=Ourense',
 'Pontevedra': 'https://www.citapreviadnie.es/citaPreviaDniExp/ObtenerOficinas.action?provincia=36&nombre_provincia=Pontevedra',
 'Asturias': 'https://www.citapreviadnie.es/citaPreviaDniExp/ObtenerOficinas.action?provincia=33&nombre_provincia=Asturias',
 'Cantabria': 'https://www.citapreviadnie.es/citaPreviaDniExp/ObtenerOficinas.action?provincia=39&nombre_provincia=Cantabria',
 'Bizkaia': 'https://www.citapreviadnie.es/citaPreviaDniExp/ObtenerOficinas.action?provincia=48&nombre_provincia=Bizkaia',
 'Gipuzkoa': 'https://www.citapreviadnie.es/citaPreviaDniExp/ObtenerOficinas.action?provincia=20&nombre_provincia=Gipuzkoa',

## Unidades

In [28]:
# driver.get(URL_DNI)
# 
# unidades_spain = []
# 
# for province_name, province_url in tqdm(provinces.items()):
#     
#     driver.get(province_url)
#     
#     listas = driver.find_elements_by_class_name('lista')
#     unidades_provincia = [extraer_datos_unidad(ud, province_name) for lista in listas for ud in lista.find_elements_by_tag_name('div')]
#     
#     unidades_spain.extend(unidades_provincia)
# 
# unidades_spain = pd.DataFrame(unidades_spain)    
# unidades_spain.to_csv('unidades_spain.csv', index = False)

  0%|          | 0/52 [00:00<?, ?it/s]

In [29]:
# sanity check - estan todas las provincias de spain?
# unidades_spain.province.unique()


# abrir csv unidades spain
unidades_spain = pd.read_csv("unidades_spain.csv")

## Geocoding

In [32]:
locator = Nominatim(user_agent="myGeocoder")

def geocode_row(row):
    location = locator.geocode(row.direccion)
    
    row['latitude'] = location.latitude if location else None
    row['longitude'] = location.longitude if location else None
    
    return row

In [33]:
unidades_spain = unidades_spain.progress_apply(geocode_row, axis=1)

  0%|          | 0/280 [00:00<?, ?it/s]

In [35]:
# unidades_spain.to_csv('unidades_spain.csv', index = False)
# unidades_spain

,url,name,title,direccion,horario,disponibilidad,province,latitude,longitude
0,https://www.citapreviadnie.es/citaPreviaDniExp...,U.D. DE CARBALLO,"U.D. DE CARBALLO RUA CERVANTES S/N, (CARBALLO)","RUA CERVANTES S/N, (CARBALLO)",LMXJV 09:00-14:30; SD sin horario,Disponibilidad de citas: Alta,A Coruña,43.213307,-8.688269
1,https://www.citapreviadnie.es/citaPreviaDniExp...,U.D. DE FERROL,"U.D. DE FERROL CALLE SAN AMARO, S/N, (FERROL)","CALLE SAN AMARO, S/N, (FERROL)",LMXJV 09:00-14:30; SD sin horario,Disponibilidad de citas: Alta,A Coruña,43.483878,-8.227722
2,https://www.citapreviadnie.es/citaPreviaDniExp...,U.D. DE A CORUÑA,"U.D. DE A CORUÑA AVENIDA PORTO DA CORUÑA, 7, (...","AVENIDA PORTO DA CORUÑA, 7, (A CORUÑA)",LMXJV 09:00-14:30; SD sin horario,Disponibilidad de citas: Alta,A Coruña,43.366112,-8.403066
3,https://www.citapreviadnie.es/citaPreviaDniExp...,U.D. DE A CORUÑA-LONZAS,U.D. DE A CORUÑA-LONZAS CALLE MÉDICO DEVESA NU...,"CALLE MÉDICO DEVESA NUÑEZ, 4, (LONZAS)",LMXJV 09:00-14:30; SD sin horario,Disponibilidad de citas: Alta,A Coruña,43.346699,-8.415388
4,https://www.citapreviadnie.es/citaPreviaDniExp...,U.D. DE RIBEIRA,"U.D. DE RIBEIRA AVENIDA DAS AIROS, 21, (RIBEIRA)","AVENIDA DAS AIROS, 21, (RIBEIRA)",LMXJV 09:00-14:30; SD sin horario,Disponibilidad de citas: Alta,A Coruña,NaN,NaN
...,...,...,...,...,...,...,...,...,...
275,https://www.citapreviadnie.es/citaPreviaDniExp...,U.D. DE GRANADA-NORTE,"U.D. DE GRANADA-NORTE CALLE LA PALMITA, 1, (GR...","CALLE LA PALMITA, 1, (GRANADA)",LMXJV 09:00-14:30/16:00-19:00; S 09:00-14:00; ...,Disponibilidad de citas: Alta,Granada,37.199508,-3.621509
276,https://www.citapreviadnie.es/citaPreviaDniExp...,U.D. DE GRANADA-SUR,"U.D. DE GRANADA-SUR BARRIO DEL ZAIDIN, PLZA L...","BARRIO DEL ZAIDIN, PLZA LAS PALOMAS, C/ CHILE ...",LMXJV 09:00-14:30; SD sin horario,Disponibilidad de citas: Alta,Granada,NaN,NaN
277,https://www.citapreviadnie.es/citaPreviaDniExp...,U.D. DE MOTRIL,"U.D. DE MOTRIL RONDA DEL MEDIODIA S/N, (MOTRIL)","RONDA DEL MEDIODIA S/N, (MOTRIL)",LMXJV 09:00-14:30; SD sin horario,Disponibilidad de citas: Alta,Granada,36.740667,-3.521366
278,https://www.citapreviadnie.es/citaPreviaDniExp...,U.D. DE CEUTA,"U.D. DE CEUTA PASEO DE COLóN, 4, (CEUTA)","PASEO DE COLóN, 4, (CEUTA)",LMXJV 09:00-14:30; SD sin horario,Disponibilidad de citas: Alta,Ceuta,35.886993,-5.312517


In [38]:
unidades_spain[:10]

,url,name,title,direccion,horario,disponibilidad,province,latitude,longitude
0,https://www.citapreviadnie.es/citaPreviaDniExp...,U.D. DE CARBALLO,"U.D. DE CARBALLO RUA CERVANTES S/N, (CARBALLO)","RUA CERVANTES S/N, (CARBALLO)",LMXJV 09:00-14:30; SD sin horario,Disponibilidad de citas: Alta,A Coruña,43.213307,-8.688269
1,https://www.citapreviadnie.es/citaPreviaDniExp...,U.D. DE FERROL,"U.D. DE FERROL CALLE SAN AMARO, S/N, (FERROL)","CALLE SAN AMARO, S/N, (FERROL)",LMXJV 09:00-14:30; SD sin horario,Disponibilidad de citas: Alta,A Coruña,43.483878,-8.227722
2,https://www.citapreviadnie.es/citaPreviaDniExp...,U.D. DE A CORUÑA,"U.D. DE A CORUÑA AVENIDA PORTO DA CORUÑA, 7, (...","AVENIDA PORTO DA CORUÑA, 7, (A CORUÑA)",LMXJV 09:00-14:30; SD sin horario,Disponibilidad de citas: Alta,A Coruña,43.366112,-8.403066
3,https://www.citapreviadnie.es/citaPreviaDniExp...,U.D. DE A CORUÑA-LONZAS,U.D. DE A CORUÑA-LONZAS CALLE MÉDICO DEVESA NU...,"CALLE MÉDICO DEVESA NUÑEZ, 4, (LONZAS)",LMXJV 09:00-14:30; SD sin horario,Disponibilidad de citas: Alta,A Coruña,43.346699,-8.415388
4,https://www.citapreviadnie.es/citaPreviaDniExp...,U.D. DE RIBEIRA,"U.D. DE RIBEIRA AVENIDA DAS AIROS, 21, (RIBEIRA)","AVENIDA DAS AIROS, 21, (RIBEIRA)",LMXJV 09:00-14:30; SD sin horario,Disponibilidad de citas: Alta,A Coruña,NaN,NaN
5,https://www.citapreviadnie.es/citaPreviaDniExp...,U.D. DE SANTIAGO DE COMPOSTELA,U.D. DE SANTIAGO DE COMPOSTELA AVENIDA RODRIGO...,"AVENIDA RODRIGO DE PADRON, 3, (SANTIAGO DE COM...",LMXJ 09:00-14:30/16:00-19:30; V 09:00-14:30; S...,Disponibilidad de citas: Alta,A Coruña,42.878537,-8.546070
6,https://www.citapreviadnie.es/citaPreviaDniExp...,U.D. DE LUGO,"U.D. DE LUGO AVENIDA DE RAMON FERREIRO, 14, (L...","AVENIDA DE RAMON FERREIRO, 14, (LUGO)",LMXJ 09:00-17:00; V 09:00-14:30; SD sin horario,Disponibilidad de citas: Alta,Lugo,43.004704,-7.554759
7,https://www.citapreviadnie.es/citaPreviaDniExp...,U.D. DE MONFORTE DE LEMOS,U.D. DE MONFORTE DE LEMOS C/ MIGUEL DE CERVANT...,"C/ MIGUEL DE CERVANTES, 1, (MONFORTE DE LEMOS)",LMXJV 09:00-14:30; SD sin horario,Disponibilidad de citas: Alta,Lugo,NaN,NaN
8,https://www.citapreviadnie.es/citaPreviaDniExp...,U.D. DE VIVEIRO,"U.D. DE VIVEIRO AVDA. RAMÓN CANOSA, S/N, (VIVE...","AVDA. RAMÓN CANOSA, S/N, (VIVEIRO)",LMXJV 09:00-14:30; SD sin horario,Disponibilidad de citas: Alta,Lugo,43.664474,-7.594516
9,https://www.citapreviadnie.es/citaPreviaDniExp...,U.D. DE OURENSE,"U.D. DE OURENSE CALLE MESTRE VIDE, 4, (OURENSE)","CALLE MESTRE VIDE, 4, (OURENSE)",LMXJ 09:00-14:30/15:00-17:00; V 09:00-14:30; S...,Disponibilidad de citas: Alta,Ourense,NaN,NaN


## Horas

El comportamiento aquí siempre es igual: el mes y día seleccionados (por defecto, el primero) no tienen enlace, los siguientes sí.

In [43]:
driver.get(unidad['url'])

First try current month, then any remaining months

In [44]:
citas = extract_hours(driver, province_name, unidad_name)

month_days = get_remaining_days(driver)
for date, date_url in tqdm(month_days.items(), desc='Current month'):
    driver.get(date_url)
    citas.extend(extract_hours(driver, province_name, unidad_name))
    
next_months = get_remaining_months(driver)
for month, month_url in next_months.items():
    driver.get(month_url)
    month_days = get_remaining_days(driver)
    for date, date_url in tqdm(month_days.items(), desc=month):
        driver.get(date_url)
        citas.extend(extract_hours(driver, province_name, unidad_name))

Current month:   0%|          | 0/9 [00:00<?, ?it/s]

Febrero:   0%|          | 0/18 [00:00<?, ?it/s]

In [45]:
citas_df = pd.DataFrame(citas)
citas_df.head()

,Dia,Hora,Citas,Provincia,Unidad
0,18/01/2022,09:36,1,A Coruña,U.D. DE CARBALLO
1,18/01/2022,09:48,2,A Coruña,U.D. DE CARBALLO
2,18/01/2022,10:00,1,A Coruña,U.D. DE CARBALLO
3,18/01/2022,10:24,2,A Coruña,U.D. DE CARBALLO
4,18/01/2022,10:48,2,A Coruña,U.D. DE CARBALLO


In [ ]:
citas_df.to_csv('citas_coruna.csv', index=False)